In [1]:
from transformers import AutoTokenizer, MBartTokenizer, MBartModel, MBartForConditionalGeneration
from datasets import load_dataset, load_from_disk
import torch
import data_prep


In [2]:
data_prep.create_datasets()

Preparing English to French
Preparing English to Spanish
Preparing English to Chinese


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/vg2565/.cache/huggingface/datasets/csv/default-05fc7f3b10286f96/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Dataset csv downloaded and prepared to /home/vg2565/.cache/huggingface/datasets/csv/default-f03cdcc0d65998f2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Saving the dataset (0/1 shards):   0%|          | 0/98 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/60 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/42 [00:00<?, ? examples/s]

In [17]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.9 MB/s eta 0:00:00a 0:00:01


In [3]:
model_mbart_cc25 = MBartModel.from_pretrained("facebook/mbart-large-cc25")
tokenizer_mbart_cc25_fr = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="en_XX", tgt_lang="fr_XX")

Some weights of the model checkpoint at facebook/mbart-large-cc25 were not used when initializing MBartModel: ['final_logits_bias']
- This IS expected if you are initializing MBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_mbart_cc25

In [4]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_datasets["train"][0]

# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# tokenized_datasets

Found cached dataset glue (/home/vg2565/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [10]:
raw_datasets["train"][0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [16]:
def source_target_sentences(source_file_path, target_file_path):
    source_sentences = []
    target_sentences = []

    with open("../WikiMatrix.en-fr.txt.en", "r") as english_sentences:
        for sentence in english_sentences.readlines():
            source_sentences.append(sentence)
            
    with open("../WikiMatrix.en-fr.txt.fr", "r") as french_sentences:
        for sentence in french_sentences.readlines():
            target_sentences.append(sentence)
    return source_sentences, target_sentences

In [17]:
english_french_en_file_path, english_french_fr_file_path = "../WikiMatrix.en-fr.txt.en", "../WikiMatrix.en-fr.txt.fr"
english_french_en_sentences, english_french_fr_sentences = source_target_sentences(english_french_en_file_path, english_french_fr_file_path)

In [23]:
from sklearn.model_selection import train_test_split

english_french_en_sentences_train, english_french_en_sentences_val, english_french_fr_sentences_train, english_french_fr_sentences_val = train_test_split(english_french_en_sentences, english_french_fr_sentences, test_size=.2)

#tokenizer_mbart_cc25(english_french_en_sentences, truncation=True)

In [24]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, source_encodings, target_encodings):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

['How is Shinjin to be Realized?\n',
 'The neural basis of always looking on the bright side.\n',
 'Awards 2010, but lost to Liquid Confidence by You Me At Six.\n',
 'The government of Niger has remained completely indifferent to our more than legitimate demands.\n',
 'David may well have been among them.\n',
 'En mort de qualité, je lui tins ce langage.\n',
 'Reistu þig við, sólin er komin á loft...\n',
 'Can coral reefs be saved?\n',
 'Jimmy Carter said it was the first time he had been denied permission to enter any country.\n',
 'Angela loves Christmas more than anything.\n',
 'It will be standard for so long as mankind is interested in mountaineering.\n',
 '(Doctor iuris) or Dr. jur.\n',
 'The bear finds Charles and Bob again.\n',
 'But Bart is easy to do.\n',
 'Matt Stone and site creator Taison Tan decided to shut the site down in April 2001 when the official site South Park Studios launched.\n',
 'You Can Be an Oni, too!!\n',
 'Now God has decided to use other methods.\n',
 "Be

In [5]:
dataset_en_fr = load_from_disk("./data/en-fr")

In [18]:
dataset_en_fr["train"]

{'en': '"Allah is the Light of the heavens and the earth.',
 'fr': "À titre d'exemple, le verset de la lumière\xa0: «\xa0Allah est la Lumière des cieux et de la terre."}

In [17]:
src_lang, trg_lang = 'en', 'fr'
tokenizer = tokenizer_mbart_cc25_fr

def preprocess_function(examples):
    inputs = [examples[src_lang]]
    targets = [examples[trg_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_dataset_en_fr = dataset_en_fr.map(preprocess_function, batched=True)

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

ValueError: too many values to unpack (expected 2)

In [20]:

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

books = load_dataset("opus_books", "en-fr")

source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

Found cached dataset opus_books (/home/vg2565/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [24]:
tokenized_books["train"]

Dataset({
    features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 127085
})